In [1]:

import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

# Complexes data Extraction

In [3]:
url = f"https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key={API_KEY}"
resp = requests.get(url = url)

In [ ]:

data = resp.json()
with open("../data/json_files/complex_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)

Saved successfully.


In [5]:
complexes = []
venues = []
for comp in data.get('complexes', []):
    id = comp.get('id')
    complexes.append({'complex_id': id, 'name': comp.get('name')})
    for venue in comp.get('venues', []):
        venues.append({'id' : venue.get('id'), 'name': venue.get('name'), 'complex_id': id, 'city_name': venue.get('city_name'), 
                     'country_name': venue.get('country_name'), 'country_code' : venue.get('country_code'), 'timezone' : venue.get('timezone')
                     })

In [6]:
complex_df = pd.DataFrame(complexes)
venues_df = pd.DataFrame(venues)

In [7]:
complex_df.to_csv('../data/raw_file/complex.csv', index = False)
venues_df.to_csv('../data/raw_file/venues.csv', index = False)

# Ranking Data Extraction

In [ ]:

url = f'https://api.sportradar.com/tennis/trial/v3/en/rankings.json?api_key={API_KEY}'
response = requests.get(url)
data = response.json()

{'generated_at': '2026-02-28T13:40:26+00:00',
 'rankings': [{'type_id': 1,
   'name': 'ATP',
   'year': 2026,
   'week': 9,
   'gender': 'men',
   'competitor_rankings': [{'rank': 1,
     'movement': 0,
     'points': 13550,
     'competitions_played': 18,
     'competitor': {'id': 'sr:competitor:407573',
      'name': 'Alcaraz, Carlos',
      'country': 'Spain',
      'country_code': 'ESP',
      'abbreviation': 'ALC'}},
    {'rank': 2,
     'movement': 0,
     'points': 10400,
     'competitions_played': 19,
     'competitor': {'id': 'sr:competitor:225050',
      'name': 'Sinner, Jannik',
      'country': 'Italy',
      'country_code': 'ITA',
      'abbreviation': 'SIN'}},
    {'rank': 3,
     'movement': 0,
     'points': 5280,
     'competitions_played': 18,
     'competitor': {'id': 'sr:competitor:14882',
      'name': 'Djokovic, Novak',
      'country': 'Serbia',
      'country_code': 'SRB',
      'abbreviation': 'DJO'}},
    {'rank': 4,
     'movement': 0,
     'points': 4555,
 

In [9]:
with open('../data/json_files/ranking.json', 'w') as f:
    json.dump(data, f, indent = 2)


In [10]:
Competitor_Rankings = []
Competitors_list = []

In [11]:
for ranking in data.get('rankings', []):
    gender = ranking.get('gender')
    tour = ranking.get('name')
    year = ranking.get('year')
    week = ranking.get('week')
    for x in ranking.get('competitor_rankings', []):
        competitors = x.get('competitor')
        Competitor_Rankings.append({'rank': x.get('rank'), 'movement': x.get('movement'), 'points': x.get('points'), 'competitions_played': x.get('competitions_played'), 'competitor_id': competitors.get('id'), 'gender': gender, 'year': year, 'week': week, 'tour': tour})
        Competitors_list.append({'id': competitors.get('id'),'name': competitors.get('name'), 'country': competitors.get('country'), 'country_code': competitors.get('country_code'), 'abbreviation': competitors.get('abbreviation')})

In [ ]:
Competitor_Rankings_df = pd.DataFrame(Competitor_Rankings)
competitor_df = pd.DataFrame(Competitors_list)

,id,name,country,country_code,abbreviation
0,sr:competitor:407573,"Alcaraz, Carlos",Spain,ESP,ALC
1,sr:competitor:225050,"Sinner, Jannik",Italy,ITA,SIN
2,sr:competitor:14882,"Djokovic, Novak",Serbia,SRB,DJO
3,sr:competitor:57163,"Zverev, Alexander",Germany,DEU,ZVE
4,sr:competitor:359602,"Musetti, Lorenzo",Italy,ITA,MUS
...,...,...,...,...,...
995,sr:competitor:214264,"Burgic, Ema",Bosnia & Herzegovina,BIH,BUR
996,sr:competitor:677415,"Dols, Sara",Spain,ESP,DOL
997,sr:competitor:315623,"Wang, Jiaqi",China,CHN,WAN
998,sr:competitor:819750,"Khayrutdinova, Ekaterina",Neutral,NaN,KHA


In [14]:
Competitor_Rankings_df.to_csv('../data/raw_file/Competitor_Rankings.csv', index = False)
competitor_df.to_csv('../data/raw_file/competitor.csv', index = False)

# Categories Table

In [15]:

url = f'https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key={API_KEY}'
response = requests.get(url)


In [ ]:
data = response.json()

with open('../data/json_files/competitions.json', 'w') as f:
    json.dump(data, f, indent = 2)

{'generated_at': '2026-02-28T13:44:12+00:00',
 'competitions': [{'id': 'sr:competition:620',
   'name': 'Hopman Cup',
   'type': 'mixed',
   'gender': 'mixed',
   'category': {'id': 'sr:category:181', 'name': 'Hopman Cup'}},
  {'id': 'sr:competition:660',
   'name': 'World Team Cup',
   'type': 'mixed',
   'gender': 'men',
   'category': {'id': 'sr:category:3', 'name': 'ATP'},
   'level': 'atp_250'},
  {'id': 'sr:competition:990',
   'name': 'ATP Challenger Tour Finals',
   'parent_id': 'sr:competition:6239',
   'type': 'singles',
   'gender': 'men',
   'category': {'id': 'sr:category:72', 'name': 'Challenger'}},
  {'id': 'sr:competition:1207',
   'name': 'Championship International Series',
   'type': 'singles',
   'gender': 'women',
   'category': {'id': 'sr:category:6', 'name': 'WTA'},
   'level': 'wta_championships'},
  {'id': 'sr:competition:2100',
   'name': 'Davis Cup',
   'type': 'mixed',
   'gender': 'men',
   'category': {'id': 'sr:category:76', 'name': 'Davis Cup'}},
  {'id'

In [17]:
categories = []
competitions = []

for comp in data.get('competitions', []):
    category_parse = comp.get('category')
    categories.append({'id': category_parse.get('id'), 'name': category_parse.get('name')})
    competitions.append({'id': comp.get('id'), 'name': comp.get('name'), 'parent_id': comp.get('parent_id'), 
                          'type': comp.get('type'), 'gender': comp.get('gender'), 'category_id': category_parse.get('id')})

In [18]:
category_df = pd.DataFrame(categories)
competition_df = pd.DataFrame(competitions)

In [19]:
category_df.to_csv('../data/raw_file/category.csv', index = False)
competition_df.to_csv('../data/raw_file/competition.csv', index = False)